In [1]:
import torch # PyTorch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Cuda varsa kullan yoksa cpu kullan. Cuda GPU'yu temsil eder.
print(device)

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])


cuda


In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)  # batch_size=4, her bir batch'te 4 resim olacak şekilde ayarlandı.

In [5]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [6]:
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False) 

In [7]:
import torch

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Conv bloğu 1
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1   = nn.BatchNorm2d(16)
        # Conv bloğu 2
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(32)
        # Conv bloğu 3
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3   = nn.BatchNorm2d(64)

        self.pool  = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.5)
        self.fc1   = nn.Linear(64 * 4 * 4, 256)
        self.fc2   = nn.Linear(256, 10)
        self.relu  = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 64*4*4)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [ ]:
# … (transform ve dataloader tanımları)
net = Net().to(device) # Modeli oluştur ve GPU'ya taşı

criterion   = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer   = torch.optim.SGD(net.parameters(), lr=0.00001,
                              momentum=0.9, weight_decay=5e-4)
scheduler   = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

for epoch in range(10):  # 10 epoch
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # veriyi al, cihaza gönder
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # geri yayılım adımları
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # loss biriktirme ve her 2000 iterasyonda ekrana yazdırma
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f"[{epoch+1}, {i+1:5d}] loss: {running_loss/2000:.3f}")
            running_loss = 0.0
    scheduler.step()

[1,  2000] loss: 2.321
[1,  4000] loss: 2.265
[1,  6000] loss: 2.221
[1,  8000] loss: 2.183
[1, 10000] loss: 2.158
[1, 12000] loss: 2.120
[2,  2000] loss: 2.089
[2,  4000] loss: 2.075
[2,  6000] loss: 2.046
[2,  8000] loss: 2.029
[2, 10000] loss: 2.024
[2, 12000] loss: 2.018
[3,  2000] loss: 1.987
[3,  4000] loss: 1.987
[3,  6000] loss: 1.970
[3,  8000] loss: 1.961
[3, 10000] loss: 1.957
[3, 12000] loss: 1.942
[4,  2000] loss: 1.932
[4,  4000] loss: 1.923
[4,  6000] loss: 1.915
[4,  8000] loss: 1.909
[4, 10000] loss: 1.900
[4, 12000] loss: 1.893
[5,  2000] loss: 1.876
[5,  4000] loss: 1.873
[5,  6000] loss: 1.873
[5,  8000] loss: 1.865
[5, 10000] loss: 1.853
[5, 12000] loss: 1.856
[6,  2000] loss: 1.845
[6,  4000] loss: 1.849
[6,  6000] loss: 1.832
[6,  8000] loss: 1.820
[6, 10000] loss: 1.812
[6, 12000] loss: 1.808
[7,  2000] loss: 1.797
[7,  4000] loss: 1.812
[7,  6000] loss: 1.800
[7,  8000] loss: 1.790
[7, 10000] loss: 1.773
[7, 12000] loss: 1.795
[8,  2000] loss: 1.777
[8,  4000] 

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test set doğruluğu: {100 * correct / total:.2f}%")

Test set doğruluğu: 44.33%


In [ ]:
# Eğitilen ağı ‘cifar_net.pth’ olarak kaydet
torch.save(net.state_dict(), 'cifar_net.pth')